# Load data

In [ ]:
with open("sentences.txt", "r") as f:
    sentences = f.read().split('\n')

# Ignore Warnings

In [ ]:
import logging
import os
import warnings

warnings.filterwarnings("ignore")
warnings.filterwarnings(action="ignore", module="tensorflow")

import tensorflow as tf

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.autograph.set_verbosity(0)

logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Syntax parsing

Лучше парсить небольшими пакетами по 1-5 предложений.

In [ ]:
from deeppavlov import build_model, configs

model = build_model("ru_syntagrus_joint_parsing", download=True)
parsed_sentences = list()
for sent in range(len(sentences[:100]) // 3 + 1):
    for parse in model(sentences[sent * 3 : (sent + 1) * 3]):
        parsed_sentences.append(parse)

# Extract time expressions

In [ ]:
from TimeExpressions.TimeProcessor import TimeProcessor

time_processor = TimeProcessor()
result = time_processor.process(sentences[:100])

# Extract events

In [ ]:
# with open("parsed_sentences.txt", "r") as f:
#     parsed_sentences = f.read().split('\n\n')

In [ ]:
%%time
import spacy
from TimeExpressions.EventExtractor import EventExtractor

event_extractor = EventExtractor()
events = event_extractor.extract(parsed_sentences[:100], result['expression'])